In [ ]:
from dotenv import load_dotenv
import os
import logging
import time
import pandas as pd
from binance import Client, exceptions
from prometheus_client import start_http_server
from requests.exceptions import ConnectionError, Timeout
from tradebot_setup import (
    initialize_metrics,
    update_metrics_on_buy,
    update_metrics_on_sell,
    buy_condition,
    sell_condition,
    process_buy,
    process_sell,
    safe_float_conversion,
    calculate_standard_deviation
)

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configure o logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Iniciar o servidor HTTP do Prometheus em uma thread separada
start_http_server(8000)

# Obtenha as chaves da API de variáveis de ambiente
api_key = os.getenv('BINANCE_API_KEY')
api_secret = os.getenv('BINANCE_API_SECRET')

if not api_key or not api_secret:
    logger.error("API Key ou API Secret não encontrada. Verifique o arquivo .env.")
    exit(1)

client = Client(api_key, api_secret, requests_params={'timeout': 20})

symbol = "BTCUSDT"
quantity = 0.0011  # A quantidade de BTC transacionada fixa
interval = '1h'  # O intervalo de tempo das velas
setup = "9.1"  # Identificador do setup de trading
stopgain_percentage = 2  # Percentual de stopgain
ema_span = 9  # Período da EMA

# Inicialize as métricas
metrics = initialize_metrics()

def get_current_balance(client, asset):
    try:
        balance_info = client.get_asset_balance(asset=asset)
        return float(balance_info['free'])
    except exceptions.BinanceAPIException as e:
        logger.error(f"Erro na API Binance ao obter saldo: {e}")
        return 0.0
    except Exception as e:
        logger.error(f"Erro inesperado ao obter saldo: {e}")
        return 0.0

def get_lot_size(client, symbol):
    try:
        info = client.get_symbol_info(symbol)
        for f in info['filters']:
            if (f['filterType'] == 'LOT_SIZE'):
                return float(f['stepSize'])
        return None
    except exceptions.BinanceAPIException as e:
        logger.error(f"Erro na API Binance ao obter LOT_SIZE: {e}")
        return None
    except Exception as e:
        logger.error(f"Erro inesperado ao obter LOT_SIZE: {e}")
        return None

def read_trade_history():
    if os.path.exists('data/trade_history.csv'):
        df = pd.read_csv('data/trade_history.csv')
        if not df.empty:
            return df
    return pd.DataFrame()

def check_last_transaction(client, symbol):
    try:
        trades = client.get_my_trades(symbol=symbol, limit=5)
        if not trades:
            return False, pd.DataFrame()
        trades_sorted = sorted(trades, key=lambda x: x['time'], reverse=True)
        last_trade = trades_sorted[0]
        is_buy = last_trade['isBuyer']
        trade_history = read_trade_history()
        return is_buy, trade_history
    except exceptions.BinanceAPIException as e:
        logger.error(f"Erro na API Binance: {e}")
        time.sleep(25)
        return check_last_transaction(client, symbol)
    except Exception as e:
        logger.error(f"Erro inesperado ao verificar a última transação: {e}")
        time.sleep(25)
        return check_last_transaction(client, symbol)

def main_loop():
    global total_trade_duration, total_trades
    is_comprado_logged = False
    is_not_comprado_logged = False
    position_maintained = False

    while True:
        try:
            is_buy, trade_history = check_last_transaction(client, symbol)
            metrics['total_trades_metric'].labels(symbol).inc()

            if is_buy and not is_comprado_logged:
                logger.info("Bot v2 iniciado - Loop de venda.")
                is_comprado_logged = True
                is_not_comprado_logged = False

            if not is_buy and not is_not_comprado_logged:
                logger.info("Bot v2 iniciado - Loop de compra.")
                is_not_comprado_logged = True

            while is_buy:
                if not position_maintained:
                    logger.info("Loop de venda - Checando condições de venda.")
                    position_maintained = True
                ticker = float(client.get_symbol_ticker(symbol=symbol)['price'])

                if not trade_history.empty:
                    stoploss = trade_history['stoploss'].iloc[-1]
                    stopgain = trade_history['stopgain'].iloc[-1]

                    if sell_condition(ticker, stoploss, stopgain):
                        balance_btc = get_current_balance(client, 'BTC')
                        lot_size = get_lot_size(client, symbol)
                        successful_sell, trade_history, buy_price, sell_price, sell_duration = process_sell(client, symbol, balance_btc, lot_size, ticker, trade_history)
                        if successful_sell:
                            is_buy = False
                            total_trade_duration += sell_duration
                            update_metrics_on_sell(metrics, ticker, symbol)
                            position_maintained = False

                            # Calcular a variação percentual
                            percent_change = ((sell_price - buy_price) / buy_price) * 100
                            logger.info(f"Venda realizada! Preço de compra: {buy_price}, Preço de venda: {sell_price}, Variação: {percent_change:.2f}%")

                    time.sleep(1)

            while not is_buy:
                if not position_maintained:
                    logger.info("Loop de compra - Checando condições de compra.")
                    position_maintained = True
                klines = client.get_klines(symbol=symbol, interval=interval, limit=50)
                data = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

                data['close'] = data['close'].apply(safe_float_conversion)
                data['low'] = data['low'].apply(safe_float_conversion)
                data['high'] = data['high'].apply(safe_float_conversion)
                data['volume'] = data['volume'].apply(safe_float_conversion)

                if data[['close', 'low', 'high', 'volume']].isnull().any().any():
                    logger.error("Dados corrompidos recebidos da API Binance.")
                    continue

                condition_met, current_price, previous_high = buy_condition(data, ema_span)
                metrics["current_price_metric"].labels(symbol).set(current_price)
                metrics["current_high_price_metric"].labels(symbol).set(data['high'].max())
                metrics["current_low_price_metric"].labels(symbol).set(data['low'].min())
                metrics["current_volume_metric"].labels(symbol).set(data['volume'].sum())
                metrics["price_standard_deviation_metric"].labels(symbol).set(calculate_standard_deviation(data['close']))

                if condition_met:
                    trade_history, buy_prices, stoploss, stopgain, potential_loss, potential_gain, buy_duration = process_buy(client, symbol, quantity, data, interval, setup, trade_history, stopgain_percentage)
                    total_trade_duration += buy_duration
                    is_buy = True
                    update_metrics_on_buy(metrics, current_price, stoploss, stopgain, potential_loss, potential_gain, symbol)
                    is_not_comprado_logged = False
                    is_comprado_logged = False
                time.sleep(1)

        except exceptions.BinanceAPIException as e:
            logger.error(f"Erro na API Binance: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except exceptions.BinanceOrderException as e:
            logger.error(f"Erro ao criar ordem na Binance: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except (ConnectionError, Timeout) as e:
            logger.error(f"Erro de conexão: {e}")
            time.sleep(25)  # Espera antes de tentar novamente
        except Exception as e:
            logger.error(f"Erro inesperado: {e}")
            time.sleep(25)  # Espera antes de tentar novamente

if __name__ == "__main__":
    main_loop()
